# Query 1
Top 5 most popular pieces of content consumed this week

Strategy: 


- The metric to measure the popularity of a piece of content is `Ads_User_id`


- Return the top 5 `Content_id` with the most `Ads_User_id` in table `Page_Impression`

Assumption:


- There are no missing data


- All the `Timestamp` are in the same time zone

```sql
SELECT Content_id, 
       COUNT(Ads_User_id) AS Popularity
FROM Page_Impression
WHERE 
  Timestamp >= (SELECT DATE_TRUNC('week',now()))
GROUP BY Content_id
ORDER BY Popularity DESC
LIMIT 5;
```

# Query 2
Number of weekly active users for the latest full week (Monday – Sunday). WAU is calculated by counting registered users with > 60 seconds dwell time between Monday-Sunday.

Strategy:


- Connect the `User_id` with `Dwell_Time` through `OAuth_id`, sum up the total `Dwell_Time` over the latest full week


- Filter those `User_id` with more than 60 seconds total `Dwell_Time`. 


- The relationship *between* table `Registered_Users` and table `OAuth_id_Service` is __one to one__, which means each entity in the table `Registered_Users` *exclusively* corresponds to only one entity in the table `OAuth_id_Service`. 


- Also, *from* table `Ad_Service_Interaction_Data` *to* table `OAuth_id_Service`, the relationship is __one__, which means any entity in table `Ad_Service_Interaction_Data` can find *only one* entity in table `OAuth_id_Service`. 


- This means __1__ `User_id` corresponds to __1__ `OAuth_id`, and __1__ `OAuth_id` corresponds to __1__ `Ads_User_id`. This also means any `User_id` or `OAuth_id` or `Ads_User_id` can identify a unique user. 


-  That means we can count `Ads_User_id` that associate to more than 60 sec of total `Dwell_Time`. There's no need to pull in table `Registered_Users` or table `OAuth_id_Service`, which can speed up the process and save the memory usage.

Assumptions:

- There are no missing data.


- All the `Timestamp` are in the same time zone.


- `User_id` is the primary key (PK) of table `Registered_Users`, so `User_id` is unique. 


- `OAuth_id` is the PK of table `OAuth_id_Service`. 


- `Dwell_Time` is in the unit of second


- `GROUP BY` works here with a 22 TB table.

```sql
SELECT COUNT(*) AS Num_WAU
FROM
(
    SELECT Ads_User_id
    FROM Ad_Service_Interaction_Data 
    WHERE 
      Timestamp >= (SELECT DATE_TRUNC('week',now()) - INTERVAL '7days') 
      AND Timestamp < (SELECT DATE_TRUNC('week',now()))
    GROUP BY Ads_User_id
    HAVING SUM(Dwell_Time) > 60
)
```

# Query 3
Top 5 pieces of content from each content type consumed this week by only active users (using the above definition)

Strategy: 


- The metric to measure the popularity of a piece of content is `Ads_User_id`


- Need to filter out those `Ads_User_id` that belong to only the active users *this week*, then find the top 5 pieces for each content type

Assumptions:


- All the assumptions from query 2


- Assume the weekly active users (WAU) is still defined by the *latest full week* data, not *this week*. While, the popularity of contents is based on *this week*'s `Ads_User_id` activities. In other words, the popularity of a piece is determined by *this week*'s `Ads_User_id` number, where `Ads_User_id` is from a weekly active users pool selected from the *latest full week*'s data.

```sql
WITH Active_Users AS
(
    SELECT Ads_User_id
    FROM Ad_Service_Interaction_Data 
    WHERE 
      Timestamp >= (SELECT DATE_TRUNC('week',now()) - INTERVAL '7days') 
      AND Timestamp < (SELECT DATE_TRUNC('week',now()))
    GROUP BY Ads_User_id
    HAVING SUM(Dwell_Time) > 60
),
    Popular_Content AS
(
    SELECT Content_id, COUNT(Ads_User_id) AS Popularity
    FROM Page_Impression
    WHERE 
      (Timestamp > (SELECT DATE_TRUNC('week',now()))) 
      AND (Ads_User_id IN Active_Users)
    GROUP BY Content_id
)

SELECT *
FROM
(
    SELECT Content_Type, 
           Content_id, 
           Popularity, 
           Rank() over (Partition BY Content_Type
                        ORDER BY Popularity DESC) AS Rank
    FROM Content_Metadata JOIN Popular_Content USING (Content_id)
)
WHERE Rank <= 5;

```